## EDA와 Feature-engineering을 통한 대출 상환여부 예측

#### reference
* Home Credit Default Risk - A Gentle Introduction <br>

https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction <br>
https://bkshin.tistory.com/entry/캐글-5-Home-Credit-Default-Risk


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# List files available
print(os.listdir("../input/"))

In [ ]:
# Training data
app_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
print('Training data shape: ', app_train.shape)
app_train.head()

In [ ]:
# Testing data features
app_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
print('Testing data shape: ', app_test.shape)
app_test.head()

--------------------------------------------------------
## Exploratory Data Analysis

## 1) Target 컬럼의 분포를 살펴보자
Target은 우리가 예측해야하는 값이다. 0이면 제때 대출금 상환 가능한것, 1이면 상황이 어려운것을 의미한다. 

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
app_train['TARGET'].astype(int).plot.hist();

* 대출을 상환할 수 있는 0값이 1보다 훨씬 많은 imbalanced data이다.

## 2) 결측치 확인
각 컬럼별 결측치 개수 및 비중 확인

In [ ]:
def missing_values_table(df):
    # 전체 결측치 개수 확인
    mis_val=df.isnull().sum()
    
    # 결측치 비중 확인
    mis_val_percent=100*df.isnull().sum()/len(df)
    
    # 결측치 개수 , 결측치 비중 테이블 만들기
    mis_val_table=pd.concat([mis_val, mis_val_percent],axis=1)
    
    # 컬럼 이름바꾸기
    mis_val_table_ren_columns=mis_val_table.rename(columns={0:'Missing Values',1:'% of Total Values'})

    # 결측치 0인 컬럼은 제외하고 정렬
    mis_val_table_ren_columns=mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1]!=0].sort_values('% of Total Values',ascending=False).round(1)

    # 요약 결과 print
    print("app_train의 전체 컬럼 개수는 "+str(df.shape[1])+"개 이다.\n"
         "그 중에서 결측치가 있는 컬럼 개수는 "+str(mis_val_table_ren_columns.shape[0])+'개 이다.')
    
    return mis_val_table_ren_columns

In [ ]:
missing_values=missing_values_table(app_train)
missing_values.head(20)

* 머신러닝 모델을 만드려면, 위의 결측치들을 채워야한다.(imputation)
* 뒷부분에서 imputation없이 결측치를 채울 수 있는 XGBoost 모델을 사용할것이다.
* 또 다른 방법으로는 결측치가 너무 많은 컬럼은 삭제할수도있지만, 해당 컬럼이 모델 성능에 도움이 될수도 있기 때문에 우선 유지하기로한다.

## 3) Column Types
int64, float64 타입은 수치형변수이고 object 타입은 범주형 변수

In [ ]:
app_train.dtypes.value_counts()

In [ ]:
# 범주형 변수에서 유니크한 값의 개수를 살펴보자
# app_train.select_dtypes('object').apply(pd.Series.nunique)
app_train.select_dtypes('object').nunique()   # apply함수 없이 가능

* 대부분의 범주형 변수는 유니크한 값이 적은것으로 보인다.
    * ORGANIZATION_TYPE 와 OCCUPATION_TYPE 는 예외

## 4) 범주형 변수 Encoding
LightGBM같은 모델을 제외하고 대부분의 머신러닝 모델은 범주형 변수를 다룰 수 없기 때문에, 이러한 범주형 변수를 encode해줘야 한다. 그 방법으로는 아래 두가지가 있음
* 1) Label encoding :
    * 범주형 변수의 개별값을 숫자로 바꿔주는 방법. 컬럼을 새로 생성하지 않음
    * 여성/남성 처럼 범주형 변수의 값이 두개일경우는 Label encoding을 사용해도 무관하지만, 그 이상일경우는 One-hot encoding을 사용하는것이 좋음
* 2) One-hot encoding :
    * 피처 값의 유형에 따라 새로운 피처를 추가해 고유 값에 해당하는 칼럼에만 1을 표시하고 나머지 칼럼에는 0을 표시하는 방법
    * One-hot encoding의 경우 범주형 변수의 유니크한값의 개수만큼 컬럼이 늘어난다는것
        * 이를 보완하기 위해 PCA같은 차원축소 방법을 사용할수도있음
        
> 이 노트북에서는 범주형변수의 유니크한 값이 2개일경우 Label encoding을 사용하고 그 이상일 경우 One-hot encoding을 사용할것다.

### Label Encoding and One-Hot Encoding
* LabelEncoder(), get_dummies() 활용

In [ ]:
le=LabelEncoder()
le_count=0

# 컬럼별로 iterate 돌기
for col in app_train:
    if app_train[col].dtype=='object':
        # 데이터타입이 object이고 값의 종류가 두개 이하일경우,
        if len(list(app_train[col].unique())) <=2:
            
            # train과 test에 동일하게 라벨인코딩을 하기위해 train기준으로 fit한값을 train,test에 동일하게 transform해줌
            le.fit(app_train[col])
            
            # train-set, test-set 둘다 Transform
            app_train[col]=le.transform(app_train[col])
            app_test[col]=le.transform(app_test[col])
            
            # 라벨인코딩을 한 컬럼이 몇개인지 카운트
            le_count+=1
print('%d columns were label encoded.' % le_count)

In [ ]:
# 위에서 Label-encoding적용 안한 나머지 범주형 변수에 One-hot encoding 적용
app_train=pd.get_dummies(app_train)
app_test=pd.get_dummies(app_test)

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

## 5) Train데이터와 Test데이터 컬럼 맞춰주기
train 데이터와 test 데이터에는 동일한 feature가 있어야 한다. <br>
train 데이터에 있는 카테고리변수의 유니크한 값 개수와 test 데이터에 있는 카테고리 변수의 유니크한 값 개수가 다른 변수들이 있어서 one-hot-encoding을 했더니, train에는 있는데 test에 없는 컬럼들이 생겨버림.<br>
<br>
따라서 test 데이터에 없고 train에만 있는 컬럼을 삭제해야됨. <br>
> 우선, train 데이터에서 TARGET 컬럼을 뽑아낸다. 
    * TARGET 컬럼은 test데이터에 없어도 train 데이터에는 반드시 있어야하기 때문에

> align() 함수의 join메소드를 inner로 적용해서 교집합으로 있는 변수만 추린다.

#### Python align() 함수
* 두 데이터 프레임에 포함 된 데이터를 변경하지 않고 두 데이터 프레임간에 행 및 / 또는 열의 배열이 동일한 지 확인할때 사용

In [ ]:
# 예시
df1 = pd.DataFrame([[1,2,3,4], [6,7,8,9]], columns=['D', 'B', 'E', 'A'], index=[1,2])
df2 = pd.DataFrame([[10,20,30,40], [60,70,80,90], [600,700,800,900]], columns=['A', 'B', 'C', 'D'], index=[2,3,4])

In [ ]:
df1

In [ ]:
df2

In [ ]:
# 두 데이터프레임에 둘다 포함되어있는 D,B,A 컬럼만 남김
a1, a2 = df1.align(df2, join='inner', axis=1)
print(a1)
print(a2)

In [ ]:
# TARGET변수는 train데이터에만 있지만 필요한 변수이기때문에 따로 빼두고나서 다시추가할것
train_labels=app_train['TARGET']

"""
두 데이터프레임에 모두 있는 컬럼들만 유지하면서 train-set과 test-set을 align한다.
즉, train 데이터와 test 데이터에 둘다 있는 컬럼들의 값만 가져오려는것
"""

app_train, app_test=app_train.align(app_test,join='inner',axis=1)

# TARGET변수 다시 추가
app_train['TARGET']=train_labels

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

## 6) Back to Exploratory Data Analysis

### 6-1) 이상치 (Anomalies)
* 이상치를 발견할 수 있는 방법중 하나는 describe()메소드로 컬럼의 통계값들을 보는것이다.

In [ ]:
# DAYS_BIRTH 컬럼에서는 이상치 없어보임
(app_train['DAYS_BIRTH'] / -365).describe()

In [ ]:
# DAYS_EMPLOYED는 이상치..
app_train['DAYS_EMPLOYED'].describe()

In [ ]:
app_train['DAYS_EMPLOYED'].plot.hist(title='Days Employment Histogram')
plt.xlabel('Days Employment');

이상치인것 같은 고객들은 따로 빼서 그들의 대출상환 비율이 그외의 고객들에비해 높거나 낮은 경향이 있는지 파악해보자


In [ ]:
# 이상치
anom=app_train[app_train['DAYS_EMPLOYED']==365243]
# 이상치 외
non_anom=app_train[app_train['DAYS_EMPLOYED']!=365243]

print('The non-anomalies default on %0.2f%% of loans' %(100*non_anom['TARGET'].mean()))
print('The anomalies default on %0.2f%% of loans' % (100 * anom['TARGET'].mean()))
print('There are %d anomalous days of employment' % len(anom))

* 이상치로 보이는 고객들이 대출을 상환하지못할 확률이 5.4%로 더 낮음.
* 이상치를 다루는 가장 안전한 방법은 결측치 채우듯이 채우는 방법
* 이 경우 모든 이상치들이 같은값을 갖고 있으므로, 다 같은 값으로 채울것이다.
* 이상값들이 중요해보이니, 머신러닝 모델에 이 이상값들을 임의로 채운것에대해 알려줄것이다.

> 결론적으로
    * 이상값을 숫자로 채우지 않고, 새로운 boolean 컬럼을 만들어서 이상값인지 아닌지를 구분할것이다.

In [ ]:
# Create an anomalous flag column
## 이상치(365243)인 값에 대해서 True , False로 구분
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# 이상치를 nan값으로 대치
app_train['DAYS_EMPLOYED'].replace({365243:np.nan},inplace=True)

app_train['DAYS_EMPLOYED'].plot.hist(title='Days Employment Histogram');
plt.xlabel('Days Employment');

In [ ]:
# test 데이터에도 train 데이터와 동일하게 작업
app_test['DAYS_EMPLOYED_ANOM']=app_test['DAYS_EMPLOYED']==365243
app_test['DAYS_EMPLOYED'].replace({365243:np.nan}, inplace=True)

# True, False로 되어있는 데이터 sum하면 True인것 개수 카운팅됨.
print('There are %d anomalies in the test data out of %d entries'%(app_test['DAYS_EMPLOYED_ANOM'].sum(), len(app_test)))

### 6-2) Correlations
이제 카테고리형 변수와 outlier를 다뤄보자. <br>
데이터를 이해하는 방법중 하나는 변수간, 그리고 target과의 상관관계를 살펴보는것이다. <br>
.corr()를 사용해서 변수간, 그리고 target변수와의 Pearson 상관관계를 살펴보자.

In [ ]:
# TARGET 변수와의 상관관계
correlations=app_train.corr()['TARGET'].sort_values()

print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

* DAYS_BIRTH 컬럼이 가장 양의 상관성이 높다. 양의 상관을 띄지만, 이 변수의 값들은 실제로 음수이다. 
    * 이 의미는 고객 나이가 많을수록 대출 상환할 가능성이 적다? 라는 해석이 나오는데 DAYS_BIRTH가 음수여서 그렇게 나타난것으로 보임. 따라서, DAYS_BIRTH에 절댓값을 취해서 다시 상관관계를 보려고 함.

### 6-3) Effect of Age on Repayment

In [ ]:
# DAYS_BIRTH의 절대값과 TARGET변수와의 상관계수
app_train['DAYS_BIRTH']=abs(app_train['DAYS_BIRTH'])
app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

* 절대값을 취해서 다시 TARGET과의 상관계수를 보니 고객의 나이가 많을수록, 대출을 제때 상환할 가능성이 높다고 나옴.

In [ ]:
plt.style.use('fivethirtyeight')

# 고객 나이에 대한 히스토그램 분포 확인
plt.hist(app_train['DAYS_BIRTH']/365, edgecolor='k',bins=25)
plt.title('Age of Client');
plt.xlabel('Age (years)');
plt.ylabel('Count');

* 위의 분포를 살펴보니 outlier없이 나이가 고르게 분포되어있는 편. 
* 이제 나이가 TARGET에 미치는 영향을 시각화해서 보기위해 KDE plot을 그려볼것이다.

> KDE plot을 사용하는 이유
* 보통 분포를 확인할 때 히스토그램을 많이 활용한다. 그런데 히스토그램은 구간을 어떻게 설정하냐에 따라 결과물이 매우 달라져서 엉뚱한 결론과 해석을 내릴 수 있음.
* 그래서 그 대안으로 커널 밀도 추정(KDE) 그래프를 많이 사용함.
    * 히스토그램 같은 분포를 곡선화해서 나타낸 그래프

In [ ]:
plt.figure(figsize=(10,8))

# 제때 대출을 상환하는 고객의 나이 plot (TARGET=0)
sns.kdeplot(app_train.loc[app_train['TARGET']==0,'DAYS_BIRTH']/365,label='target==0')

# 제때 대출을 상환하지못하는 고객의 나이 plot (TARGET=1)
sns.kdeplot(app_train.loc[app_train['TARGET']==1,'DAYS_BIRTH']/365,label='target==1')

plt.xlabel('Age(years)');
plt.ylabel('Density');
plt.title('Distribution of Ages');

* target==1(빨간색) 의 분포를 보면 20-30대에 기울어 있는것을 볼 수 있다. 이는 젊은 층일수록 대출 상환을 못할 확률이 높다고 유추할 수 있음.
* target==0일때와 1일때의 TARGET과의 분포가 상이한것으로 보아 이 변수는 머신러닝 모델에 유용하게 활용될 것으로 보인다.
* 그럼이제 나이를 나이대 별로 그룹을 나눠서 target=1(대출 상환이 어려운) 의 평균값을 살펴보자.

In [ ]:
# 최소 20 최대 70으로해서 총 10개로 그룹핑
## 결과는 20이상 25미만, 25이상 30미만,,,, 으로 그룹핑됨. 단 (,)는 포함 [,]는 미포함을 의미
np.linspace(20,70,num=11)

In [ ]:
"""
cut() 함수를 사용해서 5살 간격으로 나이대 그룹을 나눠보자. 
그다음, 각 나이대 별로 대출상환을 못하는 비율을 체크
"""

age_data=app_train[['TARGET','DAYS_BIRTH']]
age_data['YEARS_BIRTH']=age_data['DAYS_BIRTH']/365

# Bin the age data
age_data['YEARS_BINNED']=pd.cut(age_data['YEARS_BIRTH'],bins=np.linspace(20,70,num=11))
age_data.head(10)

In [ ]:
# Group by the bin and calculate averages
age_groups  = age_data.groupby('YEARS_BINNED').mean()
age_groups

In [ ]:
plt.figure(figsize=(8,8))

# Graph the age bins and the average of the target as a bar plot
plt.bar(age_groups.index.astype(str), 100*age_groups['TARGET'])

# Plot labeling
plt.xticks(rotation=75);
plt.xlabel('Age Group (years)');
plt.ylabel('Failur to Reapy(%)')
plt.title('Failure to Repay by Age Group');

* 젊은층일수록 대출을 상환하지 못하는 것으로 나타남
* 20-25세, 25-30세 30-35세는 각각 약10% 이상 대출을 상환하지 못했고, 55-60세, 60-65세, 65-70세는 5%이하로 대출을 상환하지 못했음.

### 6-4) Exterior Sources
* 음의 상관이 가장 높았던 3개의 변수 EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3 이다.
    * 이 변수들은 외부에서 가져온 정규화된 score를 나타낸다.
* 그럼, 이제 TARGET 변수와 EXT_SOURCE와의 상관관계와 EXT_SOURCE 서로간의 상관관계를 살펴보자

In [ ]:
ext_data=app_train[['TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']]
ext_data_corrs=ext_data.corr()
ext_data_corrs

In [ ]:
plt.figure(figsize=(8,6))

sns.heatmap(ext_data_corrs, cmap=plt.cm.RdYlBu_r, vmin=-0.25, annot=True, vmax=0.6)
plt.title('Correlation Heatmap');

* EXT_SOURCE와 TARGET 변수는 음의 상관성을 띄므로, EXT_SOURCE값이 증가할수록 대출 상환을 잘한다는 의미로 해석가능.
* 또한, DAYS_BIRTH 변수는 EXT_SOURCE_1 변수와 양의 상관성이 높은것으로 보아 이 score중 하나는 고객의 나이일것으로 추정된다.
* 그 다음은 각 EXT_SOURCE 를 TARGET값 별로 나눠서 분포를 살펴보자.

In [ ]:
plt.figure(figsize=(10,12))
plt.subplot(3,1,1);
sns.kdeplot(app_train.loc[app_train['TARGET']==0, 'EXT_SOURCE_1'],label='target==0')
sns.kdeplot(app_train.loc[app_train['TARGET']==1, 'EXT_SOURCE_1'],label='target==1');

In [ ]:
plt.figure(figsize=(10,12))

# iterate through the sources
for i, source in enumerate(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']):
    plt.subplot(3,1,i+1)
    
    sns.kdeplot(app_train.loc[app_train['TARGET']==0,source],label='target==0')
    sns.kdeplot(app_train.loc[app_train['TARGET']==1,source],label='target==1')
    
    plt.title('Distribution of %s by Target Value' % source)
    plt.xlabel('%s' %source);
    plt.ylabel('Density');
plt.tight_layout(h_pad=2.5)

* EXT_SOURCE_3 변수는 target값에 따라 차이가 가장 큰것으로 보인다.
* target 과의 상관계수가 그리 높지는 않지만, target이 0인지 1인지에 따라 값이 다른것으로 보아 모델에 영향을 주는 주요 변수라고 판단할 수 있음.

### 6-5) Pairs Plot
* EXT_SOURCE 와 DAYS_BIRTH 변수간의 pair plot을 그려보자. 
* pair plot은 각각의 분포를 보여줄 뿐만 아니라, 여러 변수간의 관계도 보여주는 좋은 시각화이다.

In [ ]:
# Copy the data for plotting
plot_data=ext_data.drop(columns=['DAYS_BIRTH']).copy()

# 고객 나이 컬럼 추가
plot_data['YEARS_BIRTH']=age_data['YEARS_BIRTH']

# 결측치 drop
plot_data=plot_data.dropna().loc[:100000,:]

# 두 컬럼 간의 상관관계를 계산하는 함수 작성
def corr_func(x,y,**kwargs):
    r=np.corrcoef(x,y)[0][1]
    ax=plt.gca()
    ax.annotate("r={:.2f}".format(r),
               xy=(.2, .8),
               xycoords=ax.transAxes,
               size=20)

# Create the pairgrid object
## vars = 변수명 리스트
grid=sns.PairGrid(data=plot_data, size=3, diag_sharey=False, hue='TARGET',
                 vars=plot_data.columns.drop(['TARGET']).tolist())

# 삼각형 위쪽 영역은 산점도
grid.map_upper(plt.scatter,alpha=0.2)

# 대각선은 히스토그램
grid.map_diag(sns.kdeplot)

# 삼각형 하단은 density plot
grid.map_lower(sns.kdeplot, cmap=plt.cm.OrRd_r);

plt.suptitle('Ext Source and Age Features Pairs Plot',size=32, y=1.05);

* 위의 결과에서 빨간색은 대출 상환을 못하는경우, 파란색은 대출 상환하는 경우를 나타냄.
* EXT_SOURCE_1과 YEARS_BIRTH 간의 양의 선형관계가 나타난다.


## 7) Feature Engineering
* 기존 데이터를 활용해서 새로 feature를 추가한다거나, 중요한 변수만 고른다거나, 차원을 줄이는 방식 등 여러가지 feature engineering 방법이 있음.<br>
이 노트북에서는 아래 두가지 방법의 feature engineering을 해볼것이다.

### 7-1) Polynomial Features
> 곡선 형태를 띄는 데이터를 제곱, 세제곱의 값으로 만들어서 일차방정식이 되도록 할 수 있음. 이렇게 dataset의 feature를 조정하여 다항식을 일차방정식으로 만들면 Gradient Descent 같은 알고리즘을 사용해서 학습시킬수 있음.

* 여기에서는 EXT_SOURCE_1를 제곱한값과 EXT_SOURCE_2를 제곱한 값, 그리고 EXT_SOURCE_1 x EXT_SOURCE_2 와 EXT_SOURCE_1 x EXT_SOURCE_2^2 같은 두 변수간의 곱을 새로운 변수로 만들 수 있다.이러한 변수를 상호작용항 이라고 한다.

* 어떤 변수 각각은 target변수에 영향을 미치지 않을 수 있지만, 이 두 변수를 결합했을때 target변수에 영향을 미칠 수 있다.

* 상호작용항은 통계모델에서 다수의 변수들의 효과를 파악하기위해 사용되곤한다. 하지만 머신러닝에는 자주 사용되는것을 보지는 못했다. 그래서 한번 이 상호작용항이 모델예측력에 도움이 되는지 체크해볼것이다.

* 아래 코드에서 EXT_SOURCE, DAYS_BIRTH 변수를 사용해서 polynomial feature를 만들어볼것이다.


In [ ]:
'''
우선 다항식 적용할 변수들의 null값을 imputer로 채워준다.
'''
# Make a new dataframe for polynomial features
poly_features = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH', 'TARGET']]
poly_features_test = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]

# 결측치 처리를 위해 imputer 호출
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')

# target값 따로 저장
poly_target=poly_features['TARGET']
# target값 제외한 나머지 변수 저장 
poly_features=poly_features.drop(columns=['TARGET'])

# 결측치 impute로 메꾸기 (train 데이터 기준으로 fit하고, train과 test에 둘다 transform 적용)
'''
train set 기준의 평균, 중간값 또는 최빈값으로 새로운 데이터의 null값을 채우는것
'''
poly_features=imputer.fit_transform(poly_features)
poly_features_test=imputer.transform(poly_features_test)


In [ ]:
# imputer 적용 후
poly_features

In [ ]:
poly_features_test

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Create the polynomial object with specified degree
poly_transformer=PolynomialFeatures(degree=3)

In [ ]:
# Train the polynomial features (train데이터 기준으로 fit)
poly_transformer.fit(poly_features)

# Transform the features
poly_features=poly_transformer.transform(poly_features)
poly_features_test=poly_transformer.transform(poly_features_test)
print('Polynomial Features shape: ', poly_features.shape)

* get_feature_names 메소드를 사용해서 다항식 적용한 변수이름 확인

In [ ]:
poly_transformer.get_feature_names(input_features=['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH'])[:15]

* 35개의 feature가 만들어진것을 확인.이제 이 새로운 Feature들이 target과 상관관계가 있는지 확인해보자

In [ ]:
# Create a dataframe of the features 
poly_features = pd.DataFrame(poly_features, 
                             columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                           'EXT_SOURCE_3', 'DAYS_BIRTH']))
# drop했던 TARGET변수 다시 추가
poly_features['TARGET']=poly_target

# TARGET변수와의 상관관계 확인
poly_corrs=poly_features.corr()['TARGET'].sort_values()

# Display most negative and most positive
print(poly_corrs.head(10))
print(poly_corrs.tail(5))

* 다항식으로 만들어진 몇몇 새로운 변수들은 기존 변수보다 상관관계가 더 높다.<br>
(위에서 기존 변수와 TARGET변수와의 상관관계중 가장 높았던 값이 EXT_SOURCE_3 변수가 -0.18이었는데, 두 변수 EXT_SOURCE_2 EXT_SOURCE_3 를 조합한 변수는 -0.19로 더큼)

* 실제 이 변수가 모델에 영향이 있는지는 이 변수를 넣었을때와 뺐을때 둘다 테스트해보면된다.


In [ ]:
# Put test features into dataframe
poly_features_test = pd.DataFrame(poly_features_test, 
                                  columns = poly_transformer.get_feature_names(['EXT_SOURCE_1', 'EXT_SOURCE_2', 
                                                                                'EXT_SOURCE_3', 'DAYS_BIRTH']))

# 원본 train 데이터에 새로 만든 다항변수를 merge해서 새로운 데이터셋 만들기
poly_features['SK_ID_CURR'] = app_train['SK_ID_CURR']
app_train_poly = app_train.merge(poly_features, on = 'SK_ID_CURR', how = 'left')

# 원본 test 데이터에 새로 만든 다항변수를 merge해서 새로운 데이터셋 만들기
poly_features_test['SK_ID_CURR'] = app_test['SK_ID_CURR']
app_test_poly = app_test.merge(poly_features_test, on = 'SK_ID_CURR', how = 'left')

# Align the dataframes => train데이터셋 기준으로 align 
app_train_poly, app_test_poly = app_train_poly.align(app_test_poly, join = 'inner', axis = 1)

# Print out the new shapes
print('Training data with polynomial features shape: ', app_train_poly.shape)
print('Testing data with polynomial features shape:  ', app_test_poly.shape)

### 7-2) Domain Knowledge Features
* CREDIT_INCOME_PERCENT: the percentage of the credit amount relative to a client's income
* ANNUITY_INCOME_PERCENT: the percentage of the loan annuity relative to a client's income
* CREDIT_TERM: the length of the payment in months (since the annuity is the monthly amount due
* DAYS_EMPLOYED_PERCENT: the percentage of the days employed relative to the client's age

In [ ]:
app_train_domain=app_train.copy()
app_test_domain=app_test.copy()

# train데이터에 새로운 변수 추가
app_train_domain['CREDIT_INCOME_PERCENT'] = app_train_domain['AMT_CREDIT'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['ANNUITY_INCOME_PERCENT'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_INCOME_TOTAL']
app_train_domain['CREDIT_TERM'] = app_train_domain['AMT_ANNUITY'] / app_train_domain['AMT_CREDIT']
app_train_domain['DAYS_EMPLOYED_PERCENT'] = app_train_domain['DAYS_EMPLOYED'] / app_train_domain['DAYS_BIRTH']

In [ ]:
# test데이터에 새로운 변수 추가
app_test_domain['CREDIT_INCOME_PERCENT'] = app_test_domain['AMT_CREDIT'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['ANNUITY_INCOME_PERCENT'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_INCOME_TOTAL']
app_test_domain['CREDIT_TERM'] = app_test_domain['AMT_ANNUITY'] / app_test_domain['AMT_CREDIT']
app_test_domain['DAYS_EMPLOYED_PERCENT'] = app_test_domain['DAYS_EMPLOYED'] / app_test_domain['DAYS_BIRTH']

* domain기반으로 새로 만든 변수를 TARGET별로 다른 컬러로 KDE plot을 그려보자

In [ ]:
plt.figure(figsize=(12,20))
for i, feature in enumerate(['CREDIT_INCOME_PERCENT','ANNUITY_INCOME_PERCENT','CREDIT_TERM','DAYS_EMPLOYED_PERCENT']):
    plt.subplot(4,1,i+1)
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET']==0,feature],label='target==0')
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET']==1,feature],label='target==1')
    
    plt.title('Distribution of %s by Target Value' % feature)
    plt.xlabel('%s' % feature);
    plt.ylabel('Density');
    
plt.tight_layout(h_pad=2.5)

* Target=0일때와 1일때 각 변수의 분포가 별 차이가없어서 이 feature가 유의미할지 테스트해보자

### Baseline
- 우리는 대출을 갚지 못할 확률을 예측하고자 한다. 그래서 만약 아예 모르겠다고하면 test set의 모든 관측치에 0.5라고 예측할수도있다. 이렇게하면 AUC ROC값이 0.5로 나올것이다.

## 8) Logistic Regression Implementation
- 모든 categorical 변수를 encoding한 것을 사용할것이다. 그리고 결측치를 imputation으로 채울것이고, 변수를 normalizing할것이다. 

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

# training 데이터에서 TARGET 변수 drop -> TARGET변수는 결측치처리 및 Scaling대상아니기때문에
if 'TARGET' in app_train:
    train=app_train.drop(columns=['TARGET'])
else:
    train=app_train.copy()

# 변수이름
features=list(train.columns)

# testing 데이터 복사
test=app_test.copy()

# 결측치를 median값으로 처리
imputer = SimpleImputer(strategy='median')

'''
각 Feature의 값을 일정한 범위 또는 규칙에 따르게 하기 위해서 스케일링을 사용
'''
# 각각의 변수를 0~1 사이의 값으로 만들어주는 MinMaxScaler 사용
## MinMaxScaler 클래스의 인스턴스를 만들어준다
scaler=MinMaxScaler(feature_range=(0,1))

# training 데이터에 fit
imputer.fit(train)

# training데이터와 testing데이터에 둘다 transform
## imputer 처리 하고나면 DataFrame에서 array형태로 바뀜
train=imputer.transform(train)
test=imputer.transform(test)

# Scaling
scaler.fit(train)
train=scaler.transform(train)
test=scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

- 이제 LogisticRegression을 사용해볼것인데, 오버피팅을 조절해주는 regularization 파라미터 C 를 낮춰서 세팅을 해볼것이다. 
- 우선 log_reg라는 이름으로 model을 만들어주고, .fit()을 사용해서 모델을 훈련시키고, .predict_proba()를 사용해서 testing data에 대한 값을 예측할것이다.

In [ ]:
from sklearn.linear_model import LogisticRegression

# Make the model with the specified regularization parameter
log_reg=LogisticRegression(C=0.0001)

# Train on the training data
log_reg.fit(train,train_labels)

- 예측한 결과값은 mx2 배열로 나오는데(m은 관측치 개수),첫번째 컬럼은 target이 0일 확률이고 두번째 컬럼은 target이 1일 확률이다.(따라서, 두 컬럼의 합은 1이되어야함)
- 우리가 원하는 것은 대출을 갚지 못할 확률이므로, target=1일 확률인 두번째 컬럼을 선택해야한다.

In [ ]:
# 두개의 컬럼이 나오는 것을 확인
log_reg.predict_proba(test)

In [ ]:
# Make predictions
# 두번째 컬럼 선택
log_reg_pred=log_reg.predict_proba(test)[:,1]

In [ ]:
# submission파일의 형식과 동일하게 SK_ID_CURR 와 TARGET이 들어가게 만들어준다
submit=app_test[['SK_ID_CURR']]
submit['TARGET']=log_reg_pred

submit.head()

In [ ]:
# submission 데이터를 csv file로 저장
submit.to_csv('log_reg_baseline.csv',index=False)

* LogisticRegression Score : 0.67887

## 9) Improved Model: Random Forest
* 결정트리의 단점을 보완하고 장점은 그대로 가지고 있는 모델. 대표적인 '배깅' 모델이다.(배깅(Bagging)은 bootstrap aggregating의 줄임말)
* 훈련 과정에서 구성한 다수의 결정 트리들을 랜덤하게 학습시켜 분류 또는 회귀의 결과도출에 사용함.
* 기본 결정트리는 해당 데이터에 대해 맞춰서 분류를 진행한 것이기 때문에 과적합 현상이 자주 나타나는 단점이 있는데, 랜덤포레스트는 각각의 트리가 독립적으로 학습해서 이런 단점을 개선함.


### 하이퍼파라미터 튜닝
> **n_estimators**
    * 결정트리의 갯수를 지정 (Default=10)
    * 무작정 트리 갯수를 늘린다고해서 성능 좋아지는 것 아님.시간이 걸릴 수 있음
> **random_state**
    * 랜덤하게 만들어지기 때문에 random_state를 고정해야 같은 결과를 볼 수 있음
> **verbose**
    * 실행 과정 출력 여부
> **n_jobs**
    * 적합성과 예측성을 위해 병렬로 실행할 작업 수
    * n_jobs=-1로 지정하면 컴퓨터의 모든 코어를 사용함

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest=RandomForestClassifier(n_estimators=100, random_state=50, verbose=1, n_jobs=-1)


In [ ]:
# training data에 훈련
random_forest.fit(train, train_labels)

# feature importances 추출
feature_importance_values=random_forest.feature_importances_
feature_importances=pd.DataFrame({'feature':features, 'importance':feature_importance_values})

# test 데이터에 대해 예측
predictions=random_forest.predict_proba(test)[:,1]

In [ ]:
# 제출용 dataframe만들기
submit=app_test[['SK_ID_CURR']]
submit['TARGET']=predictions

# csv 파일 저장
submit.to_csv('random_forest_baseline.csv',index=False)

* RandomForest Score :

### 9-1) Feature engineering한 데이터로 예측해보자
* (참고) 기존 train 변수에 다항변수 추가한 app_train_poly데이터가 아니라, 다항변수만 있는 poly_features로 예측한값

In [ ]:
poly_features_names = list(app_train_poly.columns)

# Impute the polynomial features
imputer = SimpleImputer(strategy = 'median')

# poly_features는 다항변수만 있는 데이터
# app_train_poly는 기존 train 데이터에 다항변수 추가한 데이터
poly_features = imputer.fit_transform(app_train_poly)
poly_features_test = imputer.transform(app_test_poly)

# Scale the polynomial features
scaler = MinMaxScaler(feature_range = (0, 1))

poly_features = scaler.fit_transform(poly_features)
poly_features_test = scaler.transform(poly_features_test)

random_forest_poly = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)

In [ ]:
# training data에 훈련시키기
random_forest_poly.fit(poly_features, train_labels)

# test데이터로 예측
predictions = random_forest_poly.predict_proba(poly_features_test)[:, 1]

In [ ]:
submit=app_test[['SK_ID_CURR']]
submit['TARGET']=predictions

submit.to_csv('random_forest_baseline_engineered.csv',index=False)

* Random Forest engineered Score :0.60467

### 9-2) Domain기반으로 만든 feature로 예측해보자

In [ ]:
app_train_domain.head()

In [ ]:
# TARGET변수제거
app_train_domain=app_train_domain.drop(columns='TARGET')
# 도메인기반으로 만든 데이터의 변수명 추출
domain_features_names=list(app_train_domain.columns)

# 결측치 처리
imputer=SimpleImputer(strategy='median')
# imputer처리 해주고나면 DataFrame 형태에서 array형태로 바뀜
domain_features=imputer.fit_transform(app_train_domain)
domain_features_test=imputer.transform(app_test_domain)

# 랜덤포레스트 모델 만들기
random_forest_domain=RandomForestClassifier(n_estimators=100, random_state=50, verbose=1, n_jobs=-1)

# 훈련시키기
random_forest_domain.fit(domain_features,train_labels)

# 변수 중요도 추출
feature_importance_values_domain=random_forest_domain.feature_importances_
feature_importances_domain=pd.DataFrame({'feature':domain_features_names,
                                        'importance':feature_importance_values_domain})

# test데이터 넣어서 예측하면 TARGET=0일 확률을 예측한 컬럼 한개와 TARGET=1일 확률을 예측한 컬럼 한개가 있는데
## 여기서 우리가 원하는것은 TARGET=1일때의 확률이므로 두번째 컬럼 선택해서 저장
predictions=random_forest_domain.predict_proba(domain_features_test)[:,1]

In [ ]:
submit=app_test[['SK_ID_CURR']]
submit['TARGET']=predictions

submit.to_csv('random_forest_baseline_domain.csv',index=False)

* Random Forest domain features :0.68354

## 10) Model Interpretation: Feature Importances

* 어떤 변수가 가장 관련이 있는지를 알기위한 가장 간단한 방법은 랜덤포레스트의 feature importances를 확인하는 것이다. EDA과정에서 변수간 상관관계분석을 통해 EXT_SOURCE 변수와 DAYS_BIRTH 변수가 중요한 변수라고 생각해볼수있다.
* 나중에는 이 feature importances를 사용해서 차원을 줄여볼것이다.

In [ ]:
def plot_feature_importances(df):
    # 중요도 높은 순으로 나열
    df=df.sort_values('importance',ascending=False).reset_index()
    
    # 중요도 전체합 대비 해당 변수의 중요도 비중변수 추가
    df['importance_normalized']=df['importance']/df['importance'].sum()
    
    # 시각화
    plt.figure(figsize=(10,6))
    ax=plt.subplot()
    
    ax.barh(list(reversed(list(df.index[:15]))), df['importance_normalized'].head(15), align='center',edgecolor='k')
    
    # Set the yticks and labels
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['feature'].head(15))
    
    # Plot labeling
    plt.xlabel('Normalized Importance'); plt.title('Feature Importances')
    plt.show()
    
    return df

In [ ]:
feature_importances

In [ ]:
# feature engineering안한 기본 변수들로 변수 중요도 추출
feature_importances_sorted=plot_feature_importances(feature_importances)

* 예상했던것 처럼 EXT_SOURCE 변수와 DAYS_BIRTH 변수가 중요변수로 나온것을 확인.
* feature importances가 모델을 해석하고 차원을 줄이는데 가장 좋은 방법이라고 할 수는 없지만, 예측을 할때 모델이 어떤 요인을 고려하는지를 이해하는데 도움이 된다.

In [ ]:
feature_importances

In [ ]:
# domain기반으로 만든 변수 대상으로 변수 중요도 도출
feature_importances_domain_sorted=plot_feature_importances(feature_importances_domain)

* 도메인 기반으로 만든 4개의 변수가 변수 중요도 top15에 포함되어있는 것을 볼 수 있다. 
> 결론적으로, 도메인기반으로 만든 변수를 모델에 포함했을때가 score가 가장 좋았음.